In [1]:
from dotenv import load_dotenv
import os
print("OPENAI_API_KEY: ", os.getenv("OPENAI_API_KEY")[:10])

OPENAI_API_KEY:  sk-proj-pw


# PromptTemplate

In [2]:
from langchain_core.prompts import PromptTemplate

In [3]:
template = "What is a good name for a company that makes {product}?"
prompt = PromptTemplate.from_template(template)

In [4]:
formatted_prompt = prompt.format(product="colorful socks")
print(formatted_prompt)

What is a good name for a company that makes colorful socks?


# ChatOpenAI

In [5]:
from langchain_openai import ChatOpenAI

In [6]:
llm = ChatOpenAI(model="gpt-4.1-nano", max_tokens=50, temperature=0.2)

In [8]:
response = llm.invoke([("human", "Hello, hwo are you?")])
print(response)
print(response.content)

content="Hello! I'm doing well, thank you. How can I assist you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 14, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_f12167b370', 'id': 'chatcmpl-Bk1xyd8uGvRNckGihV4nFg5SW7k2c', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--7b7b75e1-07f1-48a0-8d22-4828d083d0e5-0' usage_metadata={'input_tokens': 14, 'output_tokens': 16, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Hello! I'm doing well, thank you. How can I assist you today?


# Runnabel
- Runnable 인터페이스 : 실행 가능 구성 요소  
- 서로 다른 컴포넌트들이 호환되는 형태로 결할 될 수 있게 만들어 준다.

# StrOutputParser

In [9]:
from langchain_core.output_parsers import StrOutputParser

In [10]:
parser = StrOutputParser()
parsed_response = parser.invoke(response)
print(parsed_response)

Hello! I'm doing well, thank you. How can I assist you today?


# Pipeline, LCEL (LangChain Expression Language)

In [11]:
chain = prompt | llm | parser # PromptTemplate -> ChatOpenAI -> StrOutputParser

In [12]:
result = chain.invoke({"product": "coffee"})
print(result)

Sure! Here are some creative and appealing name ideas for a coffee company:

1. Brew Haven  
2. Java Junction  
3. Morning Mug  
4. Bean & Bliss  
5. Rise & Roast  
6. Caffeine Craft  
7.


# LangChain으로 토큰 사용량 추적

In [13]:
print("토큰 사용 정보 : ", response.usage_metadata)

토큰 사용 정보 :  {'input_tokens': 14, 'output_tokens': 16, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
# !uv add langchain_community

In [15]:
# 콜백을 통한 누적 토큰 추적
from langchain_community.callbacks import get_openai_callback

In [17]:
with get_openai_callback() as cb:
  res1 = llm.invoke("조치원의 오늘 날씨는 어떤지 알려줘?")
  print("응답: ", res1.content[:30], "...")
  res2 = llm.invoke("파이썬으로 리스트 만드는 방법을 예시로 알려줘.")
  print("응답: ", res2.content[:30], "...")
  

응답:  죄송하지만, 현재 실시간 날씨 정보를 제공할 수 없습니 ...
응답:  물론입니다! 파이썬에서 리스트를 만드는 방법에는 여러  ...


In [18]:
print(f"총 토큰 수 : {cb.total_tokens}")
print(f"프롬프트 토큰 : {cb.prompt_tokens}")
print(f"응답 토큰 : {cb.completion_tokens}")
print(f"총 비용 : {cb.total_cost:.6f}")

총 토큰 수 : 131
프롬프트 토큰 : 41
응답 토큰 : 90
총 비용 : 0.000040


# LangChain LLM 응답 캐싱

In [19]:
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
# InMemoryCache 설정
set_llm_cache(InMemoryCache())

In [20]:
query = "재미있는 과학자 유머 하나 알려줘."
result1 = llm.invoke(query)
print("응답1: ", result1.content)

응답1:  물론이죠! 여기 재미있는 과학자 유머 하나 알려드릴게요:

왜 천문학자는 항상 좋은 농담을 좋아할까요?  
왜냐하면, 그들은 항상 별(별) 이야기를 좋아하


In [21]:
result2 = llm.invoke(query)
print("응답2: ", result2.content)

응답2:  물론이죠! 여기 재미있는 과학자 유머 하나 알려드릴게요:

왜 천문학자는 항상 좋은 농담을 좋아할까요?  
왜냐하면, 그들은 항상 별(별) 이야기를 좋아하


In [24]:
# SQLite 를 이용 디스크 기반 캐시 설정
import os 
from langchain_community.cache import SQLiteCache

In [25]:
# 기존 캐시 DB 삭제 (실슬을 위해서!!!)
if os.path.exists(".langchain_cache.db"):
  os.remove(".langchain_cache.db")

# SQLiteCache 설정
set_llm_cache(SQLiteCache(database_path=".langchain_cache.db"))

In [26]:
query = "재미있는 과학자 유머 하나 알려줘."
result1 = llm.invoke(query)
print("응답1: ", result1.content)

응답1:  물론이죠! 여기 재미있는 과학자 유머 하나 알려드릴게요:

왜 전자는 항상 파티에 늦게 올까요?  
왜냐하면, 그는 항상 "음전하" 때문에 먼저 도착


In [27]:
result2 = llm.invoke(query)
print("응답2: ", result2.content)

응답2:  물론이죠! 여기 재미있는 과학자 유머 하나 알려드릴게요:

왜 전자는 항상 파티에 늦게 올까요?  
왜냐하면, 그는 항상 "음전하" 때문에 먼저 도착
